In [92]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re

In [111]:
url = "https://en.wikipedia.org/wiki/List_of_school_shootings_in_the_United_States"
url_19 = "https://en.wikipedia.org/wiki/List_of_school_shootings_in_the_United_States_(before_2000)"

In [178]:
response = requests.get(url_19)

In [179]:
soup = BeautifulSoup(response.text, 'html.parser')

In [180]:
table_rows = soup.find_all(name="tr")

In [189]:
ex = table_rows[36]

In [190]:
e = ex.find_all('td')

In [195]:
x = e[3].text

In [196]:
x

'?\n'

In [119]:
text_example = ex.find_all('td')

In [120]:
regex_pattern = '([A-Za-z, 0-9]+)\\n'
regex_loc = '([A-Za-z, .]*)\\n'
regex_num = '(^\d?\d?\d?\d)'
regex_des = '(?<![\\\[])[^\\\[\]]+'
regex_school = '(((([A-Z]\w+ )?([A-Z]\w+ )?([A-Z])(\w+-\w+)?(\w*\.\w*\. )?([^(a|A)t ]\w* )?([A-Z][\w+ )?([^at ]\w+\,? )?(\w\.)?(\w+\. )?)|([A-Z]\w+ )([A-Z]\w+ )?([A-Z]\w+ )?)(((e|E)lementary |(m|M)iddle |(h|H)igh )?((s|S)chool)|(c|C)ollege|(u|U)niversity|(c|C)ollegiate|(i|I)nstitute (of )? ([A-Z]\w+ )|(s|S)tadium|(C)ampus|(a|A)cademy (of )?([A-Z]\w+ )?([A-Z]\w+ ?)?([A-Z]\w+)?)|(school bus)|(([A-Z]\w+ )?((u|U)niversity |(c|C)ollege |(a|A)cademy |(i|I)nstitute )((of )(the )?(([A-Z])(\w+ ?))([A-Z]\w+ )?([A-Z]\w+)?))|(s|S)tadium|(\w+ )(Elementary)|(UCLA))'

In [121]:
# t = re.findall(regex_pattern, text_example)[0]
# print(t)

In [122]:
# t = datetime.strptime(t, "%B %d, %Y")

In [123]:
# date = re.findall(regex_pattern, text_example)[0]
# date = datetime.strptime(date, "%B %d, %Y")
# print(date)

In [124]:
school_test = re.findall(regex_school, e[4].text)

In [125]:
school_test[0]

('Bard College',
 'Bard ',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Bard ',
 '',
 '',
 'College',
 '',
 '',
 '',
 '',
 '',
 '',
 'C',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '')

In [149]:
def _extract_date(text):
    date = re.findall(regex_pattern, text)[0]
    date = datetime.strptime(date, "%B %d, %Y")
    return date

def _extract_location(text):
    location = re.findall(regex_loc, text)[0]
    return location

def _extract_death(text):
    death = re.findall(regex_num, text)[0]
    death = int(death)
    return death

def _extract_injury(text):
    injury = re.findall(regex_num, text)[0]
    injury = int(injury)
    return injury

def _extract_description(text):
    description = re.findall(regex_des, text)[0]
    return description

def _extract_school(text):
    school = re.findall(regex_school, text)
    return school

In [234]:
# Function to create dict
def parse_table_rows(table_rows):
    
    table_data_raw = {}
    table_data = {}
    
    try:
        for ix, tr in enumerate(table_rows[1:], start = 0):
            data_cols = tr.find_all("td")
            if data_cols == []:
                print(f"No Data on {ix}th entry")
            else:
                date_raw = data_cols[0].text
                location_raw = data_cols[1].text
                death_raw = data_cols[2].text
                injury_raw = data_cols[3].text
                description_raw = data_cols[4].text
                date = _extract_date(date_raw)
                location = _extract_location(location_raw)
                school = _extract_school(data_cols[4].text)
                if school == []:
                    print(f"No Data on {ix}th entry for school")
                    school = pd.NA
#                     print(school)
                else:
                    school = school[0][0]
                
                if injury_raw == "?\n":
                    injury_raw = pd.NA
                else:
                    injury = _extract_injury(injury_raw)
                    
                    if "n 1" in death_raw and "n 1" in injury_raw:
                        DIS = True
                        IIS = True
                    elif "n 1" or "n 2" in death_raw:
                        DIS = True
                        IIS = False
                    elif "n 1" in injury_raw:
                        IIS = True
                        DIS = False
                    else:
                        DIS = False
                        IIS = False
                
                death = _extract_death(death_raw)
                description = _extract_description(description_raw)

                row_data_raw = {'Date': date_raw, 
                                'Location': location_raw,
                                'Death': death_raw,
                                'Injury': injury_raw,
                                'Description': description_raw}

                row_data = {'Date': date, 
                            'Location': location, 
                            'School': school,
                            'Death': death_raw,
                            'Death_Including_Suspect': DIS,
                            'Injury': injury_raw, 
                            'Injury_Including_Suspect': IIS,
                            'Description': description}

                table_data_raw[ix] = row_data_raw
                table_data[ix] = row_data
        return table_data, table_data_raw
    except:
        print("No data")

In [235]:
ts, ts_raw = parse_table_rows(table_rows)

No Data on 0th entry
No Data on 2th entry
No Data on 3th entry for school
No Data on 6th entry
No Data on 7th entry for school
No Data on 8th entry for school
No Data on 9th entry for school
No Data on 10th entry for school
No Data on 11th entry for school
No Data on 12th entry
No Data on 13th entry for school
No Data on 15th entry for school
No Data on 16th entry for school
No Data on 19th entry for school
No Data on 20th entry
No Data on 21th entry for school
No Data on 22th entry for school
No Data on 23th entry for school
No Data on 26th entry for school
No Data on 27th entry for school
No Data on 29th entry for school
No Data on 30th entry for school
No Data on 31th entry
No Data on 33th entry for school
No Data on 37th entry for school
No Data on 38th entry
No Data on 41th entry for school
No Data on 43th entry for school
No Data on 44th entry for school
No Data on 45th entry for school
No Data on 47th entry for school
No Data on 49th entry for school
No Data on 50th entry for sc

In [236]:
df = pd.DataFrame.from_dict(ts, orient = 'index')

In [237]:
df

,Date,Location,School,Death,Death_Including_Suspect,Injury,Injury_Including_Suspect,Description
1,1840-11-12,"Charlottesville, Virginia",University of Virginia,1\n,True,0\n,False,"John Anthony Gardner Davis, a law professor at..."
3,1853-11-02,"Louisville, Kentucky",<NA>,1\n,True,0\n,False,Student Mathews Flounoy Ward shot the schoolma...
4,1856-08-16,"Florence, Alabama",The school,2\n,True,0\n,False,The schoolmaster warned students not to harm h...
5,1858-07-08,"Baltimore, Maryland",Sabbath School,1\n,True,0\n,False,The 15-year-old son of Col. John T. Farlow (Ba...
7,1860-01-21,"Todd County, Kentucky",<NA>,1\n,True,0\n,False,A son of Col. Elijah Sebree was killed by anot...
...,...,...,...,...,...,...,...,...
325,1999-04-16,"Notus, Idaho",<NA>,0\n,True,1\n,False,"15-year-old Shawn Cooper of Notus, Idaho took ..."
326,1999-04-20,"Littleton, Colorado",Columbine High School,15[n 2]\n,True,21\n,False,Columbine High School massacre: 18-year-old Er...
327,1999-05-20,"Conyers, Georgia",Heritage High School,0\n,True,6\n,False,"15-year-old student Thomas ""T.J."" Solomon Jr. ..."
328,1999-11-19,"Deming, New Mexico",Deming Middle School school,1\n,True,0\n,False,13-year-old Victor Cordova Jr. fatally shot 13...


In [215]:
df['Death'][28]

0

In [216]:
df['Description'][3]

'Student Mathews Flounoy Ward shot the schoolmaster William H.G. Butler as revenge for what Ward thought was excessive punishment of his brother the day before. Butler died, and Ward was acquitted.'

In [217]:
df['Injury'].max()

74

In [208]:
df[df['Injury'] >= 10]

,Date,Location,School,Death,Death_Including_Suspect,Injury,Injury_Including_Suspect,Description
35,1891-03-30,"Liberty, Mississippi",Parson Hill school,0,False,14,False,During a school exhibition and concert at the ...
125,1966-08-01,"Austin, Texas",University of Texas,18,False,31,False,University of Texas tower shooting: 25-year-ol...
163,1974-12-30,"Olean, New York",Olean High School,3,False,11,False,Olean High School shooting: During a two-and-a...
201,1984-02-24,"Los Angeles, California",Street Elementary School,3,False,12,False,49th Street Elementary School shooting: Tyrone...
214,1986-05-16,"Cokeville, Wyoming",Cokeville Elementary School,2,False,74,False,Cokeville Elementary School hostage crisis: 43...
238,1989-01-17,"Stockton, California",Stockton school,6,False,32,False,Stockton schoolyard shooting: 24-year-old Patr...
256,1992-05-01,"Olivehurst, California",Lindhurst High School,4,False,10,False,Lindhurst High School shooting: Former student...
259,1992-11-04,"Detroit, Michigan",Finney High School,0,False,11,False,Eleven students were hit by gunfire during a f...
319,1998-03-24,"Craighead County, Arkansas",Westside Middle School,5,False,10,False,Westside Middle School shootings: 13-year-old ...
322,1998-05-21,"Springfield, Oregon",Thurston High School,4,False,25,False,Thurston High School shooting: After killing h...


In [396]:
df

,Date,Location,School,Death,Death_Including_Suspect,Injury,Injury_Including_Suspect,Description
1,2000-02-29,"Flint, Michigan",At Buell Elementary School,1,False,0,False,Shooting of Kayla Rolland: At Buell Elementary...
2,2000-05-26,"Lake Worth, Florida",Lake Worth Middle School,1,False,0,False,"13-year-old honor student, Nathaniel Brazill w..."
3,2000-06-28,"Seattle, Washington",University of Washington Medical Center,2,True,0,False,58-year-old Director of the Division of Pathol...
4,2000-08-28,"Fayetteville, Arkansas",University of Arkansas,2,True,0,False,"36-year-old James Easton Kelly, a PhD candidat..."
5,2000-09-26,"New Orleans, Louisiana",No School Data,0,False,2,False,13 year-olds Darrel Johnson and Alfred Anderso...
...,...,...,...,...,...,...,...,...
320,2021-09-24,"Fairfield, Alabama",Fairfield High Preparatory School,0,False,1,False,A man was wounded during a football game at Fa...
321,2021-09-30,"Memphis, Tennessee",Cummings Elementary School,0,False,1,False,One student was shot in Cummings Elementary Sc...
322,2021-09-30,"Newton, Mississippi",Newton Elementary School,0,False,1,False,A 7-year-old student was injured after a gun a...
323,2021-10-01,"Houston, Texas",YES Prep Southwest school,0,False,1,False,A former student walked into the YES Prep Sout...
